<a href="https://colab.research.google.com/github/kushknows/Tensorflow/blob/master/KNN_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from sklearn import datasets
import numpy as np
import random
tf.compat.v1.disable_eager_execution()

In [0]:
iris = datasets.load_iris()

In [0]:
X = iris.data 
y = iris.target
y = list(y)

In [0]:
a = list(range(len(X)))
random.shuffle(a)                # Shuffling data

In [0]:
X_train = []
y_train = list()
X_test = []
y_test = list()

In [0]:
partition = int(0.8 * len(a))

In [0]:
train_indices = a[: partition]  # Spliting dataset into training and testing set
test_indices = a[partition :]

In [0]:
for i in train_indices:
  X_train.append(X[i])
  y_train.append(y[i])

In [0]:
for i in test_indices:
  X_test.append(X[i])
  y_test.append(y[i])


In [0]:
def min_max_normalized(data):
  col_max = np.max(data, axis=0)
  col_min = np.min(data, axis=0)
  return np.divide(data - col_min, col_max - col_min )

In [0]:
X_train = min_max_normalized(X_train)
X_test = min_max_normalized(X_test)

In [0]:
sess = tf.compat.v1.Session()

In [0]:
sess.run(tf.compat.v1.global_variables_initializer())

In [0]:
input_data = tf.compat.v1.placeholder(dtype=tf.float32, shape= [None, len(X[0])])
output_data = tf.compat.v1.placeholder(dtype = tf.float32, shape= [None, 1])
test_input_datapoint = tf.compat.v1.placeholder(dtype=tf.float32, shape= [len(X[0])])

In [0]:
k = 3

In [0]:
#Calculate nearest distance btw test data pt and training data pt
distance = tf.negative(tf.reduce_sum(
    tf.abs(tf.subtract(input_data, test_input_datapoint)), axis = 1))
values, indices = tf.compat.v1.nn.top_k(distance, k= k, sorted= False)

In [0]:
# top kth neighbours list
neighbours_list = []
for i in range(k):
  neighbours_list.append(output_data[indices[i]])

In [0]:
nearest_neighbors = neighbours_list

In [0]:
cnt = 0

In [69]:
for i in range(X_test.shape[0]):
  fd = {
      input_data : X_train,
      output_data : np.matrix(y_train).T,
      test_input_datapoint : X_test[i]
  }
  neighbours = sess.run(nearest_neighbors, feed_dict = fd)
  neighbor_dict = dict()
  for neighbor in neighbours :
    if neighbor[0] in neighbor_dict:
      neighbor_dict[neighbor[0]] += 1 # No. of times particular neighbor is occuring
    else:
      neighbor_dict[neighbor[0]] = 1  # this loop is used to identify the neighbors

  best_neighbor =None 
  best_neighbor_count = 0
  for neighbor in neighbor_dict:
    if neighbor_dict[neighbor] > best_neighbor_count:
      best_neighbor = neighbor
      best_neighbor_count = neighbor_dict[neighbor]   # this loop is used to find the class
  best_neighbor = int(best_neighbor)
  if best_neighbor == y_test[i]:
    cnt += 1  
print(100*cnt/len(y_test))

96.66666666666667


In [81]:
neighbor_dict

{0.0: 3}